# Exploring Meteorite Dataset from NASA
Link to dataset on kaggle: https://www.kaggle.com/datasets/nasa/meteorite-landings


In [ ]:
# Import necessary packages for EDA
import numpy as np # for linear algebra
import pandas as pd # for data processing
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
import plotly.subplots as ps
import plotly.graph_objects as go
import folium as fl  # for geoviz
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition

In [ ]:
# Read csv file to pandas DataFrame
# We will call it Metorite DataFrame (mdf)
mdf = pd.read_csv('meteorite-landings.csv')
mdf.shape

In [ ]:
mdf.head()

### 1. Data cleaning
Information about dataset:
 - ***name***: the name of the meteorite (typically a location, often modified with a number, year, composition, etc)  
 - ***id***: a unique identifier for the meteorite
 - ***nametype***: one of:
    - *valid*: a typical meteorite
    - *relict*: a meteorite that has been highly degraded by weather on Earth
 - ***recclass***: the class of the meteorite; one of a large number of classes based on physical, chemical, and other characteristics (see the Wikipedia article on meteorite classification for a primer)
 - ***mass***: the mass of the meteorite, in grams
 - ***fall***: whether the meteorite was seen falling, or was discovered after its impact; one of:
    - *Fell*: the meteorite's fall was observed
    - *Found*: the meteorite's fall was not observed
 - ***year***: the year the meteorite fell, or the year it was found (depending on the value of fell)
 - ***reclat***: the latitude of the meteorite's landing
 - ***reclong***: the longitude of the meteorite's landing
 - ***GeoLocation***: a parentheses-enclose, comma-separated tuple that combines reclat and reclong

In [ ]:
# Check for missing values
mdf.isnull().sum()

In [ ]:
mdf['GeoLocation'].value_counts()

In [ ]:
# What will we do with missing coordinates?
# There is a lot of coordiantes (0, 0) so they also should be treated as missing values
# so lets assign (0, 0) to all missing values to group all of them together
mdf.loc[mdf['GeoLocation'].isnull(), 'reclat'] = 0
mdf.loc[mdf['GeoLocation'].isnull(), 'reclong'] = 0

# Fill in GeoLocation
mdf.loc['GeoLocation'] = mdf[['reclat', 'reclong']].apply(tuple, axis = 1)

# Drop rows with missing values for year and mass column
mdf.dropna(inplace=True, axis=0)

In [ ]:
mdf.isnull().sum()

In [ ]:
# Check for duplicates
mdf.duplicated().sum()

In [ ]:
# Check for odd values
mdf.describe()

From the table above we see, that we have wierd years in our dataset, for example 2101. Lets filter them out (dataset is from 2016), we also will be interested only in relatively recent data, this why we will filter out the years before 1800.

In [ ]:
# Filter out weird years
mdf = mdf[(mdf['year'] <= 2016) & (mdf['year'] >= 1800)]

# Filter out weird locations
mdf = mdf[(mdf['reclat'] <= 180) & (mdf['reclat'] >= -180)]

In [ ]:
# Lets transform mass from grams to kg
mdf['mass'] = mdf['mass']/1000

In [ ]:
# How many unique meteorite classes we have in our dataset
len(mdf['recclass'].unique())

So we have 422 unique classes of meteorites, we can't deal with it. If explore more ([Wikipedia](https://en.wikipedia.org/wiki/Meteorite_classification)) we will find that meteorites can  be distributed into 3 larger groups:
1. Stony;
2. Stony-iron;
3. Iron.

So lets redistribute our data to this groups.
![Meteorite_class](Meteorite_Classification_after_Weissberg_McCoy_Krot_2006_Stony_Iron.png)

In [ ]:
# One can notice that names of all classes from Iron group start with I
# Lets add a new column 'recgroup' 
mdf.loc[mdf['recclass'].str.startswith('I'), 'recgroup'] = 'Iron'

# All Stony group classes start with L, C, H, R, E, K. We also can see that all martian meteorites are stony ones
stony_letters = ('L', 'C', 'H', 'R', 'E', 'K')
mdf.loc[mdf['recclass'].str.startswith(stony_letters), 'recgroup'] = 'Stony'
mdf.loc[mdf['recclass'].str.startswith('Martian'), 'recgroup'] = 'Stony'
mdf.loc[mdf['recclass'].str.startswith('Stone'), 'recgroup'] = 'Stony'

# Stony-iron group consists only of Mesosiderite and Pallasite
mdf.loc[mdf['recclass'].str.startswith('Mesosiderite'), 'recgroup'] = 'Stony-iron'
mdf.loc[mdf['recclass'].str.startswith('Pallasite'), 'recgroup'] = 'Stony-iron'

In [ ]:
# How many values left ungrouped?
mdf.isnull().sum()

In [ ]:
# From what classes they are?
mdf[mdf['recgroup'].isnull()]['recclass'].unique()

Not so many classes left ungrouped, lets figure them out.
 - **Acapulcoite**, **Angrite**, **Aubrite**, **Diogenite**, **Brachinite**, **Ureilite**, **Winonaite** are from Stony group;
 - **Fusion crust**, **OC**, **Achondrite-prim**, **Achondrite-ung** can't be grouped.


In [ ]:
stony = ('Acapulcoite', 'Angrite', 'Aubrite', 'Diogenite', 'Brachinite', 'Ureilite', 'Winonaite')
mdf.loc[mdf['recclass'].str.startswith(stony), 'recgroup'] = 'Stony'


In [ ]:
# How many left ungrouped in percantage
ungrouped_perc = mdf['recgroup'].isnull().sum()/mdf.shape[0]*100
ungrouped_perc

Only around 0.3% left ungrouped, this is great!  Lets mark them as 'Unknown'.

In [ ]:
unknown = ('Fusion crust', 'OC', 'Achondrite-prim', 'Achondrite-ung')
mdf.loc[mdf['recclass'].str.startswith(unknown), 'recgroup'] = 'Unknown'

In [ ]:
# Check if we missed something
mdf.isnull().sum()

### 2. EDA with visualization
After the dataset was cleaned let's visualize the data to better understand what we have.

In [ ]:
# How many meteorites in each group?
group_count = mdf['recgroup'].value_counts()
sns.barplot(x = group_count.index, y = group_count)
plt.xlabel("Meteorite groups")
plt.ylabel('Number of meteorites')
plt.title('Group count')
plt.show()

As one can see from this bar plot, most of the meteorites are from the Stony group, a few from the Iron group, and even fewer from the Stony-iron group.

In [ ]:
# Count number of falls per year
number_of_falls = mdf[['year']].groupby('year', as_index=False).value_counts()
number_of_falls

In [ ]:
# Firstly, we want to see yearly destribution
sns.lineplot(data = number_of_falls, x = 'year', y = 'count')
plt.ylabel('Falls per year')
plt.xlabel('Year')
plt.title('Yearly destribution of falls')
plt.show()

We see that the number of falls increased significantly, starting around 1960. One possible explanation could be the fact that we have become better at detecting meteorites due to the development of communication technology.

In [ ]:
# Now, lets look at mass destribution
sns.scatterplot(data = mdf, x = 'mass', y = 'id', hue = 'fall', size=.01)
plt.xscale('log')
plt.ylabel('Meteorite id')
plt.xlabel('Mass, (kg)')
plt.title('Mass scatter')
plt.show()

We see that most meteorites have a mass in range from few grams to few kg. Furthermore, the dominant part was found without being seen falling.

In [ ]:
# Lets plot mass destribution
sns.histplot(data = mdf[mdf['mass'] <= 10], x = 'mass', bins=100)
plt.xscale('log')
plt.xlabel('Mass, (kg)')
plt.title('Mass destribution (for < 10kg)')
plt.show()

From this histogram we see that most meteorites have mass smaller than 100g. 

In [ ]:
px.pie(names=['Non-observed', 'Observed'], 
       values=mdf[['fall']].value_counts(), 
       title='Meteorite fall percantage %')

As mentioned previously, the fall of over 97% of all found meteorites was not observed.

In [ ]:
mdf['nametype'].value_counts()

In [ ]:
px.pie(mdf[['nametype']].value_counts(), 
       names = list(mdf['nametype'].value_counts().index), 
       values = 'count', title = 'Valid vs Relict')

Almost all meteorites were found before they can be degraded by weather conditions. This could be due to several reasons. For one, they were found not long after the fall or their landing area provided the shelter from weathering. But clear answer requires further investigation.

In [ ]:
# Now lets see where meteorites were found on map
# Firstly, we will start with creating a terrain map
meteorite_map = fl.Map(location=[0, 0], tiles="Stamen Terrain", zoom_start=2.4)

# Marker for each meteorite, it displays Year, coordinates and meteorite class
locations = list(mdf[['reclat','reclong']].set_index('reclat').itertuples())
popups = [f"Date: {int(date)}, coordinates: ({lat}, {long}), Meteorite class: {mtype}" 
          for (date, lat, long, mtype) 
          in mdf[['year', 'reclat', 'reclong', 'recclass']].set_index('year').itertuples()]
marker_cluster = MarkerCluster(locations=locations, popups=popups).add_to(meteorite_map)

In [ ]:
# Lets add mouse posiition to help investigate meteorites locations
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(meteorite_map)


In [ ]:
meteorite_map

Surprisingly enough, from the map above, we can see that Antarctica has the highest number of found meteorites (if we exclude meteorites with (0,0) coordinates), and they were discovered in three locations. If we investigate further, we will find that these locations are:
 1. Queen Alexandra Range
 2. Yamato Glacier
 3. Grove Mountains

Why is that? One [article](https://www.geographyrealm.com/why-are-there-more-meteorites-in-antarctica/) suggests:       
"Meteorites in Antarctica are more visible because the environmental conditions are favorable for the preservation and retrieval of these space rocks. The arid and cold Antarctic environment helps to preserve these rocks. ... In addition, researchers have identified meteorite hotspots known as “meteorite stranding zones”. These are areas where the local geology, the flow of the ice, and climate conditions promote the aggregation of meteorites at the surface of the blue ice."

![Antarctica meteorite map](map-antarctic-meteorites-2022-nasa.png)  

From this picture from NASA ([link](https://earthobservatory.nasa.gov/images/149554/finding-meteorite-hotspots-in-antarctica)) we see that meteorites are mostly found in a mountain area.

In [ ]:
# Lets verify our assumption about location names using our dataset
# 'name' column in the dataset contains location names, lets extract them
to_replace = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '(', ')', ' ']
for char_replace in to_replace:
    mdf['name'] = mdf['name'].str.replace(char_replace, '')
mdf['name'] = mdf['name'].str.lower()
top_names = mdf['name'].value_counts().head(20)

In [ ]:
# Top 20 names in the dataset
plt.figure(figsize=(10,10))
ax = sns.barplot(x = top_names.index, y = top_names)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.xlabel('Names')
plt.ylabel('')
plt.title('Top 20 meteorite names')
plt.show()

Bar plot above confirmes our suggestion. Indeed, 3 in a top 4 locations are from Antarctica: Yamato, Queen Alexandra Range and Grove Mountains. Furthermore, fifth name from Antarctica as well, Elephant Moraine located near Ross Ice Shelf. Lets mark this four locations on the map.

In [ ]:
# Antarctica map creation with markers
antarctica_map = fl.Map(location=(-73.913347, 67.28871), tiles="Stamen Terrain", zoom_start=3)

# Adds markers for top 4 meteorite locations
top_loc = {'Yamato Glacier': (-71.416667, 35.583333), 
           'Queen Alexandra Range': (-84, 168), 
           'Grove Mountains': (-72.75, 75), 
           'Elephant Moraine': (-76.283333, 157.333333)}
for loc in top_loc.keys():
    fl.Marker(location=top_loc[loc], 
              popup = loc, 
              icon = fl.Icon(color = 'red')).add_to(antarctica_map)

# Adds markers for each meteorite found in Antarctica      
locations = list(mdf[mdf['reclat'] <= -68][['reclat','reclong']].set_index('reclat').itertuples())
popups = [f"Date: {int(date)}, coordinates: ({lat}, {long}), Meteorite class: {mtype}" 
          for (date, lat, long, mtype)
          in mdf[mdf['reclat'] <= -68][['year', 'reclat', 'reclong', 'recclass']].set_index('year').itertuples()]

antarctic_markers = MarkerCluster(locations=locations, popups=popups).add_to(antarctica_map)

In [ ]:
antarctica_map

In [ ]:
# Are there any difference in a group meteorites found in Antarctica? Or there is more meteorites of a one kind?
plt.figure(figsize=(12,10))
ant_met = mdf[mdf['reclat'] <= -68]['recgroup'].value_counts()
sns.barplot(x = ant_met.index, y = ant_met)
plt.xlabel('Meteorite groups')
plt.ylabel('Number of meteorites')
plt.title('Groups distribution in Antarctica')
plt.show()

It doesn't seem to be any difference in group distribution. Most of the meteorites are from the Stony group, but let's compare numbers side by side.

In [ ]:
fig = ps.make_subplots(rows = 1, cols = 2, specs=[[{"type": "domain"}, {"type": "domain"}]])

fig.add_trace(go.Pie(labels = ant_met.index, values = ant_met, 
       title = 'Antarctica',
       hole = .4
       ), row = 1, col = 1)

fig.add_trace(go.Pie(labels = group_count.index, values = group_count,
       title = 'World',
       hole = .4
       ), row = 1, col = 2)
fig.update_layout(title_text = 'Group distribution: Antarctica vs World')

In this comparison, we see that there is a difference in meteorites found in Antarctica compared to the rest of the world. More than 99% are from the Stony group, whereas in the world, the total is around 97%. However, the largest difference is in the Iron group, with 0.58% in Antarctica compared to 2.23% worldwide.

In [ ]:
mdf.dtypes

In [ ]:
# Extracts numeric columns and converts fall and recgroup to numeric
mdf_num = pd.get_dummies(data = mdf, columns = ['fall', 'recgroup'])
mdf_num.drop(['name', 'id', 'nametype', 'recclass', 'GeoLocation'], axis=1, inplace = True)

In [ ]:
corr_matrix = mdf_num.corr()
fig = plt.figure(figsize = (12,10))
sns.heatmap(data = corr_matrix, annot = True)
plt.show()


Except for the obvious anti-correlation between dummies from the 'recgroup' column, we can see some notable correlations between the columns 'recgroup_Iron', 'recgroup_Stony', 'fall_Found', 'fall_Fell', and 'year', 'reclat'. So let's investigate this correlations further.

In [ ]:
# We will start with correlations between 'year' and 'recgroup' columns
year_group = mdf[['year', 'recgroup']]
fig, axes = plt.subplots(2,1, figsize = (12,10))
sns.histplot(data = year_group[year_group['recgroup']=='Stony'], x = 'year', hue = 'recgroup', ax = axes[0], kde = True)
sns.histplot(data = year_group[year_group['recgroup']!='Stony'], x = 'year', hue = 'recgroup', ax = axes[1], kde = True)
plt.show()

As seen in the pictures above, the correlation between the columns 'year' and 'recgroup' is obvious. With time, the number of meteorites found increases, as does the number of meteorites in each group. Interestingly enough, the number of meteorites in the Iron group remains pretty steady.

In [ ]:
# Now lets look at the correlation between columns 'year' and 'fall'
fall_group = mdf[['year', 'fall']]
fig, axes = plt.subplots(2,1, figsize = (12,10))
sns.histplot(data = fall_group[fall_group['fall'] == 'Found'], x = 'year', ax = axes[0], kde = True)
sns.histplot(data = fall_group[fall_group['fall'] == 'Fell'], x = 'year', ax = axes[1], kde = True)
axes[0].set_title('Non-observed meteorites distribution')
axes[1].set_title('Observed meteorites distribution')
plt.show()

Here we see the same situation where the number of meteorites found increases with time, creating a correlation between these two columns. Another interesting observation is that number of observed meteorites doesn't increase that much with time.

This leads to a question: Do the distribution patterns of meteorite groups for observed meteorites differ from those of non-observed ones? Lets figure this out!

In [ ]:
# Firstly, we separate observed and non-observed meteorites
fell_met = mdf[mdf['fall'] == 'Fell']['recgroup'].value_counts()
found_met = mdf[mdf['fall'] == 'Found']['recgroup'].value_counts()

# Secondly, lets look at numbers by plotting a pie-charts for each one
fig = ps.make_subplots(rows = 1, cols = 2, specs=[[{"type": "domain"}, {"type": "domain"}]])

fig.add_trace(go.Pie(labels = fell_met.index, values = fell_met, 
       title = 'Observed',
       hole = .4
       ), row = 1, col = 1)

fig.add_trace(go.Pie(labels = found_met.index, values = found_met,
       title = 'Non-observed',
       hole = .4
       ), row = 1, col = 2)
fig.update_layout(title_text = 'Group distribution: Observed vs Non-observed')

We observe no significant difference in the group distribution between observed and non-observed meteorites. In both cases, over 90% belong to the Stony group. However, the percentage of observed meteorites from the Iron group is higher – 4.42% compared to 2.19% for non-observed meteorites.

We have investigated correlations concerning the 'year' column. Now, let's shift our focus to the 'reclat' column.

In [ ]:
mdf['GeoLocation'].value_counts()

In [ ]:
# Connection between 'reclat' and 'recgroup' columns
#reclat_group = mdf[mdf['GeoLocation'] != (0.0,0.0)][['reclat', 'recgroup']]
reclat_group = mdf[(mdf[['reclat', 'reclong']].apply(tuple, axis=1) != (0.0, 0.0))][['reclat', 'recgroup']]
fig, axes = plt.subplots(2,1, figsize = (12,10))
sns.histplot(data = reclat_group[year_group['recgroup']=='Stony'], x = 'reclat', hue = 'recgroup', ax = axes[0], kde = True)
sns.histplot(data = reclat_group[year_group['recgroup']!='Stony'], x = 'reclat', hue = 'recgroup', ax = axes[1], kde = True)
axes[0].set_xlabel('Latitude')
axes[1].set_xlabel('Latitude')
plt.show()